# Team

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
from sqlalchemy import create_engine
from geojson import Feature, FeatureCollection, Point

In [5]:
Coordinates = pd.read_csv('Resources/world_country_latitude_and_longitude_values.csv')
Coordinates

,Country,Latitude,Longitude
0,Andorra,42.50,1.50
1,United Arab Emirates,24.00,54.00
2,Afghanistan,33.00,65.00
3,Antigua and Barbuda,17.05,-61.80
4,Anguilla,18.25,-63.17
...,...,...,...
235,Yemen,15.00,48.00
236,Mayotte,-12.83,45.17
237,South Africa,-29.00,24.00
238,Zambia,-15.00,30.00


In [6]:
Team_data = pd.read_csv('Resources/Footballteams.csv')
Team_data['Country']=Team_data['Country'].apply(lambda x: x.strip())
Team_data

,Team,Tournament,Goals,Shots pg,yellow_cards,red_cards,Possession%,Pass%,AerialsWon,Rating,Country
0,Manchester City,Premier League,83,15.8,46,2,60.8,89.4,12.8,7.01,England
1,Bayern Munich,Bundesliga,99,17.1,44,3,58.1,85.5,12.9,6.95,Germany
2,Paris Saint-Germain,Ligue 1,86,15.0,73,7,60.1,89.5,9.5,6.88,France
3,Barcelona,LaLiga,85,15.3,68,2,62.4,89.7,10.6,6.87,Spain
4,Real Madrid,LaLiga,67,14.4,57,2,57.7,87.7,11.8,6.86,Spain
...,...,...,...,...,...,...,...,...,...,...,...
93,Sheffield United,Premier League,20,8.5,73,3,43.0,76.9,19.1,6.46,England
94,Crotone,Serie A,45,9.5,85,4,47.2,80.4,12.7,6.43,Italy
95,Benevento,Serie A,40,11.0,90,5,44.2,77.7,13.4,6.43,Italy
96,Dijon,Ligue 1,25,9.2,75,5,46.9,80.0,14.3,6.42,France


In [7]:
Team_data_Final = pd.merge(Team_data,Coordinates,on = "Country")
Team_data_Final

,Team,Tournament,Goals,Shots pg,yellow_cards,red_cards,Possession%,Pass%,AerialsWon,Rating,Country,Latitude,Longitude
0,Manchester City,Premier League,83,15.8,46,2,60.8,89.4,12.8,7.01,England,54.00,-2.00
1,Manchester United,Premier League,73,13.8,64,1,54.5,84.8,14.5,6.85,England,54.00,-2.00
2,Aston Villa,Premier League,55,13.7,63,4,49.1,78.6,19.4,6.84,England,54.00,-2.00
3,Chelsea,Premier League,58,14.6,49,3,58.6,87.0,15.2,6.83,England,54.00,-2.00
4,Liverpool,Premier League,68,16.0,40,0,59.0,85.7,14.3,6.82,England,54.00,-2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Fiorentina,Serie A,47,9.8,86,5,46.8,81.0,14.2,6.52,Italy,42.83,12.83
94,Spezia,Serie A,52,10.2,92,5,51.6,81.1,14.4,6.50,Italy,42.83,12.83
95,Parma Calcio 1913,Serie A,39,10.4,91,1,48.4,82.5,16.9,6.49,Italy,42.83,12.83
96,Crotone,Serie A,45,9.5,85,4,47.2,80.4,12.7,6.43,Italy,42.83,12.83


In [8]:
# columns used for constructing geojson object
features = Team_data_Final.apply(
    lambda row: Feature(geometry=Point((float(row['Longitude']), float(row['Latitude'])))),
    axis=1).tolist()

# all the other columns used as properties
properties = Team_data_Final.drop(['Latitude', 'Longitude'], axis=1).to_dict('records')

# whole geojson object
feature_collection = FeatureCollection(features=features, properties=properties)

In [17]:
with open('Team_Data.geojson', 'w', encoding='utf-8') as f:
    json.dump(feature_collection, f, ensure_ascii=False)

In [4]:
engine = create_engine('sqlite:///football_team_data.db')

In [5]:
Team_data.to_sql('premier_league_team_data', con=engine, if_exists='replace', index=False)

In [6]:
engine.execute("SELECT * FROM premier_league_team_data").fetchall()

[('Manchester City', 'Premier League', 83, 15.8, 46, 2, 60.8, 89.4, 12.8, 7.01, 'England '),
 ('Bayern Munich', 'Bundesliga', 99, 17.1, 44, 3, 58.1, 85.5, 12.9, 6.95, 'Germany '),
 ('Paris Saint-Germain', 'Ligue 1', 86, 15.0, 73, 7, 60.1, 89.5, 9.5, 6.88, 'France '),
 ('Barcelona', 'LaLiga', 85, 15.3, 68, 2, 62.4, 89.7, 10.6, 6.87, 'Spain'),
 ('Real Madrid', 'LaLiga', 67, 14.4, 57, 2, 57.7, 87.7, 11.8, 6.86, 'Spain'),
 ('Manchester United', 'Premier League', 73, 13.8, 64, 1, 54.5, 84.8, 14.5, 6.85, 'England '),
 ('Juventus', 'Serie A', 77, 15.7, 76, 6, 55.4, 88.3, 11.4, 6.85, 'Italy '),
 ('Aston Villa', 'Premier League', 55, 13.7, 63, 4, 49.1, 78.6, 19.4, 6.84, 'England '),
 ('Borussia Dortmund', 'Bundesliga', 75, 14.6, 43, 1, 57.5, 85.5, 12.8, 6.84, 'Germany '),
 ('Atletico Madrid', 'LaLiga', 67, 12.1, 100, 0, 51.8, 83.1, 14.4, 6.84, 'Spain'),
 ('Atalanta', 'Serie A', 90, 16.3, 66, 3, 53.5, 83.5, 16.8, 6.84, 'Italy '),
 ('Chelsea', 'Premier League', 58, 14.6, 49, 3, 58.6, 87.0, 15.2, 

In [11]:
chelsea_players = pd.read_csv('Resources/chelsea_players.csv')
chelsea_players_Final = pd.merge(chelsea_players,Coordinates,how='left', left_on=['Nationality'], right_on=['Country'])

# columns used for constructing geojson object
features = chelsea_players_Final.apply(
    lambda row: Feature(geometry=Point((float(row['Longitude']), float(row['Latitude'])))),
    axis=1).tolist()

# all the other columns used as properties
properties = chelsea_players_Final.drop(['Latitude', 'Longitude'], axis=1).to_dict('records')

# whole geojson object
feature_collection = FeatureCollection(features=features, properties=properties)

with open('chelsea_players_Final.geojson', 'w', encoding='utf-8') as f:
    json.dump(feature_collection, f, ensure_ascii=False)

chelsea_players_Final

,Player,Club,Nationality,Country,Latitude,Longitude
0,Edouard Mendy,Chelsea FC,Senegal,Senegal,14.00,-14.00
1,Kepa,Chelsea FC,Spain,Spain,40.00,-4.00
2,Kurt Zouma,Chelsea FC,France,France,46.00,2.00
3,Andreas Christensen,Chelsea FC,Denmark,Denmark,56.00,10.00
4,Antonio Rüdiger,Chelsea FC,Germany,Germany,51.00,9.00
5,Ethan Ampadu,Chelsea FC,Wales,NaN,NaN,NaN
6,Thiago Silva,Chelsea FC,Brazil,Brazil,-10.00,-55.00
7,Jake Clarke-Salter,Chelsea FC,England,England,54.00,-2.00
8,Ben Chilwell,Chelsea FC,England,England,54.00,-2.00
9,Marcos Alonso,Chelsea FC,Spain,Spain,40.00,-4.00


In [ ]:
manchester_players = pd.read_csv('Resources/manchester_players.csv')
psg_players = pd.read_csv('Resources/psg_players.csv')
madrid_players = pd.read_csv('Resources/real_madrid_players.csv')
chelsea_players

In [14]:
manchester_players = pd.read_csv('Resources/manchester_players.csv')
manchester_players_Final = pd.merge(manchester_players,Coordinates,how='left', left_on=['Nationality'], right_on=['Country'])

# columns used for constructing geojson object
features = manchester_players_Final.apply(
    lambda row: Feature(geometry=Point((float(row['Longitude']), float(row['Latitude'])))),
    axis=1).tolist()

# all the other columns used as properties
properties = manchester_players_Final.drop(['Latitude', 'Longitude'], axis=1).to_dict('records')

# whole geojson object
feature_collection = FeatureCollection(features=features, properties=properties)

with open('manchester_players_Final.geojson', 'w', encoding='utf-8') as f:
    json.dump(feature_collection, f, ensure_ascii=False)

manchester_players_Final

,Player,Club,Nationality,Country,Latitude,Longitude
0,Ederson,Manchester City,Brazil,Brazil,-10.00,-55.00
1,Zack Steffen,Manchester City,United States,United States,38.00,-97.00
2,Rúben Dias,Manchester City,Portugal,Portugal,39.50,-8.00
3,Aymeric Laporte,Manchester City,Spain,Spain,40.00,-4.00
4,Nathan Aké,Manchester City,Netherlands,Netherlands,52.50,5.75
5,John Stones,Manchester City,England,England,54.00,-2.00
6,Philippe Sandler,Manchester City,Netherlands,Netherlands,52.50,5.75
7,Oleksandr Zinchenko,Manchester City,Ukraine,Ukraine,49.00,32.00
8,Benjamin Mendy,Manchester City,France,France,46.00,2.00
9,João Cancelo,Manchester City,Portugal,Portugal,39.50,-8.00


In [15]:
psg_players = pd.read_csv('Resources/psg_players.csv')
psg_players_Final = pd.merge(psg_players,Coordinates,how='left', left_on=['Nationality'], right_on=['Country'])

# columns used for constructing geojson object
features = psg_players_Final.apply(
    lambda row: Feature(geometry=Point((float(row['Longitude']), float(row['Latitude'])))),
    axis=1).tolist()

# all the other columns used as properties
properties = psg_players_Final.drop(['Latitude', 'Longitude'], axis=1).to_dict('records')

# whole geojson object
feature_collection = FeatureCollection(features=features, properties=properties)

with open('psg_players_Final.geojson', 'w', encoding='utf-8') as f:
    json.dump(feature_collection, f, ensure_ascii=False)

psg_players_Final

,Player,Club,Nationality,Country,Latitude,Longitude
0,Keylor Navas,Paris Saint-Germain,Costa Rica,Costa Rica,10.00,-84.00
1,Sergio Rico,Paris Saint-Germain,Spain,Spain,40.00,-4.00
2,Marcin Bulka,Paris Saint-Germain,Poland,Poland,52.00,20.00
3,Alexandre Letellier,Paris Saint-Germain,France,France,46.00,2.00
4,Garissone Innocent,Paris Saint-Germain,Haiti,Haiti,19.00,-72.42
5,Denis Franchi,Paris Saint-Germain,Italy,Italy,42.83,12.83
6,Marquinhos,Paris Saint-Germain,Brazil,Brazil,-10.00,-55.00
7,Presnel Kimpembe,Paris Saint-Germain,France,France,46.00,2.00
8,Thilo Kehrer,Paris Saint-Germain,Germany,Germany,51.00,9.00
9,Abdou Diallo,Paris Saint-Germain,Senegal,Senegal,14.00,-14.00


In [18]:
madrid_players = pd.read_csv('Resources/real_madrid_players.csv')
madrid_players_Final = pd.merge(madrid_players,Coordinates,how='left', left_on=['Nationality'], right_on=['Country'])

# columns used for constructing geojson object
features = madrid_players_Final.apply(
    lambda row: Feature(geometry=Point((float(row['Longitude']), float(row['Latitude'])))),
    axis=1).tolist()

# all the other columns used as properties
properties = madrid_players_Final.drop(['Latitude', 'Longitude'], axis=1).to_dict('records')

# whole geojson object
feature_collection = FeatureCollection(features=features, properties=properties)

with open('manchester_players_Final.geojson', 'w', encoding='utf-8') as f:
    json.dump(feature_collection, f, ensure_ascii=False)

madrid_players_Final

,Player,Club,Nationality,Coordinates,Country,Latitude,Longitude
0,Thibaut Courtois,Real Madrid,Belgium,"50.503887, 4.469936",Belgium,50.83,4.00
1,Andriy Lunin,Real Madrid,Ukraine,"48.379433,-31.16557999",Ukraine,49.00,32.00
2,Raphaël Varane,Real Madrid,France,"46.227638, 2.213749",France,46.00,2.00
3,Éder Militão,Real Madrid,Brazil,"-14.235004, -51.92528",Brazil,-10.00,-55.00
4,Nacho Fernández,Real Madrid,Spain,"40.463667, -3.74922",Spain,40.00,-4.00
5,Ferland Mendy,Real Madrid,France,"46.227638, 2.213749",France,46.00,2.00
6,Marcelo,Real Madrid,Brazil,"-14.235004, -51.92528",Brazil,-10.00,-55.00
7,Daniel Carvajal,Real Madrid,Spain,"40.463667, -3.74922",Spain,40.00,-4.00
8,Álvaro Odriozola,Real Madrid,Spain,"40.463667, -3.74922",Spain,40.00,-4.00
9,Casemiro,Real Madrid,Brazil,"-14.235004, -51.92528",Brazil,-10.00,-55.00


In [5]:
chelsea_group = chelsea_players.groupby("Nationality").agg({"Nationality":["count"]})
#nationality_group.reset_index(inplace=True)
#nationality_group.columns = ["Nationality","Count"]
#Sorted_nationalities = nationality_group.sort_values(by = "Count", ascending = False)
#Sorted_nationalities

In [6]:
manchester_players

,Player,Club,Nationality
0,Ederson,Manchester City,Brazil
1,Zack Steffen,Manchester City,United States
2,Rúben Dias,Manchester City,Portugal
3,Aymeric Laporte,Manchester City,Spain
4,Nathan Aké,Manchester City,Netherlands
5,John Stones,Manchester City,England
6,Philippe Sandler,Manchester City,Netherlands
7,Oleksandr Zinchenko,Manchester City,Ukraine
8,Benjamin Mendy,Manchester City,France
9,João Cancelo,Manchester City,Portugal


In [7]:
psg_players

,Player,Club,Nationality
0,Keylor Navas,Paris Saint-Germain,Costa Rica
1,Sergio Rico,Paris Saint-Germain,Spain
2,Marcin Bulka,Paris Saint-Germain,Poland
3,Alexandre Letellier,Paris Saint-Germain,France
4,Garissone Innocent,Paris Saint-Germain,Haiti
5,Denis Franchi,Paris Saint-Germain,Italy
6,Marquinhos,Paris Saint-Germain,Brazil
7,Presnel Kimpembe,Paris Saint-Germain,France
8,Thilo Kehrer,Paris Saint-Germain,Germany
9,Abdou Diallo,Paris Saint-Germain,Senegal


In [8]:
madrid_players

,Player,Club,Nationality
0,Thibaut Courtois,Real Madrid,Belgium
1,Andriy Lunin,Real Madrid,Ukraine
2,Raphaël Varane,Real Madrid,France
3,Éder Militão,Real Madrid,Brazil
4,Nacho Fernández,Real Madrid,Spain
5,Ferland Mendy,Real Madrid,France
6,Marcelo,Real Madrid,Brazil
7,Daniel Carvajal,Real Madrid,Spain
8,Álvaro Odriozola,Real Madrid,Spain
9,Casemiro,Real Madrid,Brazil


In [9]:
frames = [chelsea_players, manchester_players, psg_players,madrid_players]
all_nationalities = pd.concat(frames)
all_nationalities

,Player,Club,Nationality
0,Edouard Mendy,Chelsea FC,Senegal
1,Kepa,Chelsea FC,Spain
2,Kurt Zouma,Chelsea FC,France
3,Andreas Christensen,Chelsea FC,Denmark
4,Antonio Rüdiger,Chelsea FC,Germany
...,...,...,...
18,Rodrygo,Real Madrid,Brazil
19,Lucas Vázquez,Real Madrid,Spain
20,Karim Benzema,Real Madrid,France
21,Luka Jovic,Real Madrid,Serbia


In [10]:
nationality_group = all_nationalities.groupby(["Club","Nationality"]).agg({"Nationality":["count"]})
nationality_group.reset_index(inplace=True)
nationality_group.columns = ["Club","Nationality","Count"]
Sorted_nationalities = nationality_group.sort_values(by = "Club", ascending = False)
Sorted_nationalities

,Club,Nationality,Count
46,Real Madrid,Uruguay,1
41,Real Madrid,Germany,1
37,Real Madrid,Brazil,5
38,Real Madrid,Croatia,1
39,Real Madrid,Dominican Republic,1
40,Real Madrid,France,3
36,Real Madrid,Belgium,2
42,Real Madrid,Norway,1
43,Real Madrid,Serbia,1
44,Real Madrid,Spain,6


In [12]:
html_table = nationality_group.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Club</th>\n      <th>Nationality</th>\n      <th>Count</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Chelsea FC</td>\n      <td>Belgium</td>\n      <td>2</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Chelsea FC</td>\n      <td>Brazil</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Chelsea FC</td>\n      <td>Croatia</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Chelsea FC</td>\n      <td>Denmark</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Chelsea FC</td>\n      <td>England</td>\n      <td>9</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Chelsea FC</td>\n      <td>France</td>\n      <td>3</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Chelsea FC</td>\n      <td>Germany</td>\n      <td>3</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td

In [18]:
nationality_group.to_html('Data_table.html')